In [6]:
import os
from dotenv import load_dotenv
import json
import logging
import asyncio
import nest_asyncio  # Important for running asyncio in Jupyter notebooks
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime

# LlamaIndex imports
from llama_index.core import Settings, VectorStoreIndex, Document, StorageContext, load_index_from_storage

from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.workflow import Context, InputRequiredEvent, HumanResponseEvent
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.readers.file import PDFReader
from llama_index.core.node_parser import SentenceSplitter

In [7]:
# Load environment variables from .env file
load_dotenv()

# Apply nest_asyncio to allow asyncio to work in Jupyter notebooks
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("InsuranceAgentSystem")
logger.setLevel(logging.DEBUG)

# for Azure OpenAI models
api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
gpt_api_version = os.getenv('AZURE_GPT_API_VERSION')
embedding_api_version = os.getenv('AZURE_EMBEDDING_API_VERSION')

class InsuranceAgentSystem:
    """Main class for the Insurance Agent System implementing Stage 4 (Rank or Reject)"""
    
    def __init__(self):
        """Initialize the system with LLM, embeddings and guidelines"""
        self.setup_llm_and_embeddings()
        self.setup_guidelines_vector_store()
        self.create_agents()
        
    def setup_llm_and_embeddings(self):
        """Configure the LLM and embedding models"""
        self.llm = AzureOpenAI(
            model="gpt-4o",
            deployment_name="gpt-4o",
            api_key=api_key,
            azure_endpoint=azure_endpoint,
            api_version=gpt_api_version,
            temperature=0.1,  # Lower temperature for consistent decisions
            timeout=60,
        )
        
        self.embed_model = AzureOpenAIEmbedding(
            model="text-embedding-ada-002",
            deployment_name="text-embedding-ada-002",
            api_key=api_key,
            azure_endpoint=azure_endpoint,
            api_version=embedding_api_version,
        )
        
        # Set as default models
        Settings.llm = self.llm
        Settings.embed_model = self.embed_model
        
        logger.info("LLM and embedding models initialized")
    
    def setup_guidelines_vector_store(self):
        """Initialize the guidelines vector store from the PDF"""
        try:
            # Define paths
            pdf_path = "data/UW Commercial Insurance Manual.pdf"
            persist_dir = "storage/guidelines_index"
            
            # Check if index already exists
            if os.path.exists(persist_dir) and os.listdir(persist_dir):
                logger.info(f"Loading existing index from {persist_dir}")
                storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
                self.guidelines_index = load_index_from_storage(storage_context, index_cls=VectorStoreIndex)
            else:
                # Load and process PDF if needed
                logger.info(f"Loading PDF document from {pdf_path}")
                pdf_reader = PDFReader()
                documents = pdf_reader.load_data(pdf_path)
                logger.info(f"Loaded {len(documents)} documents from PDF")
                
                # Configure settings with node parser
                Settings.node_parser = SentenceSplitter(
                    chunk_size=1024,
                    chunk_overlap=200,
                    separator=" ",
                    paragraph_separator="",
                )
                
                # Create index
                logger.info("Creating vector store index...")
                self.guidelines_index = VectorStoreIndex.from_documents(documents)
                logger.info("Successfully created index")
                
                # Persist the index
                os.makedirs(persist_dir, exist_ok=True)
                logger.info(f"Persisting index to {persist_dir}")
                self.guidelines_index.storage_context.persist(persist_dir=persist_dir)
                logger.info("Successfully persisted index")
            
            # Create retriever
            self.guidelines_retriever = self.guidelines_index.as_retriever(similarity_top_k=3)

            # Initialize the specialized retrievers and assessment tools
            self.guidelines_helper = GuidelinesRetriever(self.guidelines_retriever)
            self.assessment_tools = AssessmentTools(self.guidelines_helper)
            
            logger.info("Guidelines vector store and assessment tools initialized successfully")
        except Exception as e:
            logger.error(f"Error setting up guidelines vector store: {str(e)}")
            import traceback
            logger.error(f"Traceback: {traceback.format_exc()}")
            self.guidelines_index = None
            self.guidelines_retriever = None
            self.guidelines_helper = None
            self.assessment_tools = None
            logger.warning("Using mock guidelines data as fallback")
    
    def create_agents(self):
        """Create all the specialized agents for the workflow"""
        # The actual agent creation happens at the end of this method to keep all tools in scope
        # Create all the agents with their tools
        
        # Hazard Agent Tools
        async def evaluate_hazard_classification(ctx: Context, property_data: Dict[str, Any]) -> str:
            """
            Evaluates if property complies with hazard classification guidelines.
            
            Args:
                property_data: Dictionary containing property details
                    
            Returns:
                Compliance assessment with issues if any
            """
            logger.info("Starting hazard classification compliance check")
            current_state = await ctx.get("state")
            
            try:
                # Extract property characteristics
                building_type = property_data.get("building_type", "")
                construction = property_data.get("construction", "")
                occupancy = property_data.get("occupancy", "")
                year_built = property_data.get("year_built", 0)
                
                # Get guidelines info from context
                guidelines_info = current_state.get("guidelines_retrieved", {}).get("hazard", "")
                
                # Check compliance against guidelines
                compliance_issues = []
                
                # Check building type eligibility
                ineligible_building_types = ["nightclub", "bar", "tavern", "disco"]
                for ineligible in ineligible_building_types:
                    if ineligible in building_type.lower():
                        compliance_issues.append(f"Building type '{building_type}' may not be eligible for coverage")
                
                # Check construction compliance
                if "wood shake" in construction.lower() or "shake roof" in construction.lower():
                    compliance_issues.append("Buildings with wood shake roofs are ineligible for coverage")
                
                # Check building age
                building_age = datetime.now().year - year_built if year_built > 0 else 30
                if building_age > 35:
                    compliance_issues.append(f"Building age ({building_age} years) exceeds maximum guideline of 35 years")
                
                # Create assessment result
                assessment_result = {
                    "compliant": len(compliance_issues) == 0,
                    "issues": compliance_issues,
                    "guidelines_referenced": guidelines_info[:200] + "..." if len(guidelines_info) > 200 else guidelines_info
                }
                
                # Store in context
                if "assessment_results" not in current_state:
                    current_state["assessment_results"] = {}
                
                current_state["assessment_results"]["hazard"] = assessment_result
                
                # Track completion
                if "completed_assessments" not in current_state:
                    current_state["completed_assessments"] = []
                
                if "hazard" not in current_state["completed_assessments"]:
                    current_state["completed_assessments"].append("hazard")
                
                await ctx.set("state", current_state)
                
                # Return assessment summary
                if assessment_result["compliant"]:
                    return "Hazard assessment completed. Property complies with hazard guidelines."
                else:
                    issues_text = "; ".join(compliance_issues)
                    return f"Hazard assessment completed. Property does not comply with guidelines: {issues_text}"
            except Exception as e:
                logger.error(f"Error in hazard classification: {str(e)}")
                return f"Error in hazard classification: {str(e)}"
        
        async def query_hazard_guidelines(ctx: Context, query: str) -> str:
            """
            Queries the underwriting guidelines for hazard-related information.
            
            Args:
                query: The search query for hazard-related guidelines
                
            Returns:
                Relevant guidelines information
            """
            logger.info(f"Querying hazard guidelines with: {query}")
            
            if not hasattr(self, 'guidelines_retriever') or self.guidelines_retriever is None:
                return "Guidelines retriever not available. Using default guidelines."
            
            try:
                if hasattr(self, 'guidelines_helper') and self.guidelines_helper:
                    building_type = ""
                    construction = ""
                    
                    # Try to extract building type and construction from query
                    if "building" in query.lower() and "with" in query.lower():
                        parts = query.lower().split("with")
                        if len(parts) >= 2:
                            building_parts = parts[0].split("for")
                            if len(building_parts) >= 2:
                                building_type = building_parts[1].strip()
                            construction = parts[1].strip()
                    
                    # Use enhanced guideline retrieval
                    guidelines_info = await self.guidelines_helper.get_hazard_guidelines(
                        building_type or query, 
                        construction or "general"
                    )
                else:
                    # Fallback to basic retrieval
                    retrieval_results = self.guidelines_retriever.retrieve(query)
                    guidelines_info = "".join([node.text for node in retrieval_results])
                
                # # Store in context
                # current_state = await ctx.get("state")
                # if "guidelines_retrieved" not in current_state:
                #     current_state["guidelines_retrieved"] = {}
                
                # current_state["guidelines_retrieved"]["hazard"] = guidelines_info
                # await ctx.set("state", current_state)
                
                # NEW: Log actual guidelines content
                logger.debug(f"Retrieved hazard guidelines:{guidelines_info}")

                # Store in context
                current_state = await ctx.get("state")
                current_state.setdefault("guidelines_retrieved", {})["hazard"] = guidelines_info
                await ctx.set("state", current_state)

                
                logger.info("Successfully retrieved hazard guidelines")
                return f"Retrieved hazard guidelines: {guidelines_info}"
            except Exception as e:
                logger.error(f"Error querying hazard guidelines: {str(e)}")
                return "Error retrieving hazard guidelines. Using default assessment only."
        
        # Vulnerability Agent Tools
        async def query_vulnerability_guidelines(ctx: Context, query: str) -> str:
            """
            Queries the underwriting guidelines for vulnerability-related information.
            
            Args:
                query: The search query for vulnerability-related guidelines
                
            Returns:
                Relevant guidelines information
            """
            logger.info(f"Querying vulnerability guidelines with: {query}")
            
            if not hasattr(self, 'guidelines_retriever') or self.guidelines_retriever is None:
                return "Guidelines retriever not available. Using default guidelines."
            
            try:
                if hasattr(self, 'guidelines_helper') and self.guidelines_helper:
                    building_type = ""
                    building_age = 0
                    
                    # Try to extract building type and age from query
                    if "for" in query.lower() and "buildings" in query.lower():
                        parts = query.lower().split("for")
                        if len(parts) >= 2:
                            building_parts = parts[1].split("buildings")
                            if len(building_parts) >= 1:
                                building_type = building_parts[0].strip()
                    
                    # See if there's age info
                    if "years" in query.lower():
                        parts = query.lower().split("years")
                        if len(parts) >= 1:
                            for word in parts[0].split()[::-1]:
                                if word.isdigit():
                                    building_age = int(word)
                                    break
                    
                    # Use enhanced guideline retrieval
                    guidelines_info = await self.guidelines_helper.get_vulnerability_guidelines(
                        building_type or query, 
                        building_age or 20
                    )
                else:
                    # Fallback to basic retrieval
                    retrieval_results = self.guidelines_retriever.retrieve(query)
                    guidelines_info = "".join([node.text for node in retrieval_results])
                
                # Store in context
                current_state = await ctx.get("state")
                current_state.setdefault("guidelines_retrieved", {})["vulnerability"] = guidelines_info
                await ctx.set("state", current_state)
                
                logger.info("Successfully retrieved vulnerability guidelines")
                return f"Retrieved vulnerability guidelines: {guidelines_info}"
            except Exception as e:
                logger.error(f"Error querying vulnerability guidelines: {str(e)}")
                return "Error retrieving vulnerability guidelines. Using default assessment only."

        async def evaluate_vulnerability(ctx: Context, security_data: Dict[str, Any]) -> str:
            """
            Evaluates if property security measures comply with vulnerability guidelines.
            
            Args:
                security_data: Dictionary containing security details
                    
            Returns:
                Compliance assessment with issues if any
            """
            logger.info("Starting vulnerability compliance check")
            current_state = await ctx.get("state")
            
            try:
                # Extract security characteristics
                has_sprinklers = security_data.get("sprinklers", False)
                alarm_system = security_data.get("alarm_system", "None")
                building_type = security_data.get("building_type", "")
                stories = security_data.get("stories", 1)
                
                # Get guidelines info from context
                guidelines_info = current_state.get("guidelines_retrieved", {}).get("vulnerability", "")
                
                # Check compliance against guidelines
                compliance_issues = []
                
                # Check sprinkler requirements
                if not has_sprinklers and stories > 2:
                    compliance_issues.append("Buildings over 2 stories must be fully sprinklered")
                
                # Check alarm system requirements
                if alarm_system == "None" and "retail" in building_type.lower():
                    compliance_issues.append("Retail buildings require at minimum a local alarm system")
                
                # Check vacancy if available
                vacancy = security_data.get("vacancy_percentage", 0)
                if vacancy > 25:
                    compliance_issues.append(f"Buildings with more than 25% vacancy are ineligible for coverage")
                
                # Check protection class if available
                protection_class = security_data.get("protection_class", 0)
                if protection_class > 6:
                    compliance_issues.append(f"Property must be within Protection Classes 1-6")
                
                # Create assessment result
                assessment_result = {
                    "compliant": len(compliance_issues) == 0,
                    "issues": compliance_issues,
                    "guidelines_referenced": guidelines_info[:200] + "..." if len(guidelines_info) > 200 else guidelines_info
                }
                
                # Store in context
                if "assessment_results" not in current_state:
                    current_state["assessment_results"] = {}
                
                current_state["assessment_results"]["vulnerability"] = assessment_result
                
                # Track completion
                if "completed_assessments" not in current_state:
                    current_state["completed_assessments"] = []
                
                if "vulnerability" not in current_state["completed_assessments"]:
                    current_state["completed_assessments"].append("vulnerability")
                
                await ctx.set("state", current_state)
                
                # Return assessment summary
                if assessment_result["compliant"]:
                    return "Vulnerability assessment completed. Security measures comply with guidelines."
                else:
                    issues_text = "; ".join(compliance_issues)
                    return f"Vulnerability assessment completed. Security measures do not comply with guidelines: {issues_text}"
            except Exception as e:
                logger.error(f"Error in vulnerability assessment: {str(e)}")
                return f"Error in vulnerability assessment: {str(e)}"

        async def query_decision_guidelines(ctx: Context, query: str) -> str:
            """
            Queries the underwriting guidelines for decision-making information.
            
            Args:
                query: The search query for decision-related guidelines
                
            Returns:
                Relevant guidelines information
            """
            logger.info(f"Querying decision guidelines with: {query}")
            
            if not hasattr(self, 'guidelines_retriever') or self.guidelines_retriever is None:
                return "Guidelines retriever not available. Using default guidelines."
            
            try:
                if hasattr(self, 'guidelines_helper') and self.guidelines_helper:
                    # Use enhanced guideline retrieval
                    submission_data = (await ctx.get("state")).get("submission_data", {})
                    guidelines_info = await self.guidelines_helper.get_decision_guidelines(submission_data)
                else:
                    # Fallback to basic retrieval
                    retrieval_results = self.guidelines_retriever.retrieve(query)
                    guidelines_info = "".join([node.text for node in retrieval_results])
                
                # Store in context
                current_state = await ctx.get("state")
                current_state.setdefault("guidelines_retrieved", {})["decision"] = guidelines_info
                await ctx.set("state", current_state)
                
                logger.info("Successfully retrieved decision guidelines")
                return f"Retrieved decision guidelines: {guidelines_info}"
            except Exception as e:
                logger.error(f"Error querying decision guidelines: {str(e)}")
                return "Error retrieving decision guidelines. Using default assessment logic only."

        async def make_decision(ctx: Context) -> str:
            """
            Makes a binary decision (Pass/Fail/Human) based on compliance with guidelines,
            requesting human intervention when needed.
            
            Returns:
                Decision outcome with explanation
            """
            logger.info("Starting decision making process")
            current_state = await ctx.get("state")
            
            # Check if all assessments are complete
            completed_assessments = current_state.get("completed_assessments", [])
            required_assessments = ["hazard", "vulnerability"]
            
            missing_assessments = [a for a in required_assessments if a not in completed_assessments]
            
            if missing_assessments:
                logger.info(f"Cannot make decision yet. Missing assessments: {missing_assessments}")
                return f"Cannot make decision yet. The following assessments are still pending: {', '.join(missing_assessments)}"
            
            # Check if we already have a final decision (to prevent infinite loops)
            if current_state.get("decision", {}).get("final", False):
                decision = current_state.get("decision", {}).get("outcome", "UNKNOWN")
                reason = current_state.get("decision", {}).get("reason", "No reason provided")
                
                decision_messages = {
                    "PASS": "Submission approved to proceed.",
                    "HUMAN_REVIEW": "Submission requires human review.",
                    "FAIL": "Submission rejected."
                }
                
                logger.info(f"Using existing final decision: {decision}")
                return f"Final decision already made: {decision_messages.get(decision, decision)}. Reason: {reason}."
            
            # Get assessment results
            assessment_results = current_state.get("assessment_results", {})
            submission_data = current_state.get("submission_data", {})
            property_details = submission_data.get("property_details", {})
            
            # Check for missing critical information
            missing_info = []
            
            if not property_details.get("address", ""):
                missing_info.append("Property address is missing")
            
            if not property_details.get("building_type", ""):
                missing_info.append("Building type is missing")
            
            if not property_details.get("construction", ""):
                missing_info.append("Construction details are missing")
            
            # Check for sanctions (special case for immediate rejection)
            address = property_details.get("address", "").lower()
            insured_name = submission_data.get("insured_name", "").lower()
            
            sanctions_issues = []
            sanctioned_countries = ["iran", "north korea", "syria", "cuba"]
            
            for country in sanctioned_countries:
                if country in address or country in insured_name:
                    sanctions_issues.append(f"Potential sanctions issue: {country} identified in submission")
            
            # Make decision
            if sanctions_issues:
                decision = "FAIL"
                reason = f"Sanctions issues: {'; '.join(sanctions_issues)}"
                requires_human_review = True  # Sanctions always need human confirmation
                
            elif missing_info:
                decision = "HUMAN_REVIEW"
                reason = f"Missing information: {'; '.join(missing_info)}"
                requires_human_review = True
                
            elif not assessment_results.get("hazard", {}).get("compliant", False):
                decision = "FAIL"
                reason = f"Hazard compliance issues: {'; '.join(assessment_results.get('hazard', {}).get('issues', []))}"
                requires_human_review = True  # Non-compliance should be human-verified
                
            elif not assessment_results.get("vulnerability", {}).get("compliant", False):
                decision = "FAIL"
                reason = f"Vulnerability compliance issues: {'; '.join(assessment_results.get('vulnerability', {}).get('issues', []))}"
                requires_human_review = True  # Non-compliance should be human-verified
                
            else:
                decision = "PASS"
                reason = "All compliance checks passed"
                requires_human_review = False  # Clean passes don't need human verification
            
            # Create initial decision object
            decision_result = {
                "outcome": decision,
                "reason": reason,
                "requires_human_review": requires_human_review,
                "human_reviewed": False,
                "final": not requires_human_review  # Only mark as final if human review not needed
            }
            
            # Store for context
            current_state["decision"] = decision_result
            await ctx.set("state", current_state)
            
            # Request human feedback if needed
            if requires_human_review:
                logger.info(f"Decision requires human review: {decision} - {reason}")
                
                # Check if human feedback was already received (prevents loops)
                if current_state.get("human_feedback_received", False):
                    feedback_comment = current_state.get("human_feedback_comment", "No feedback provided")
                    logger.info(f"Using previously received human feedback: {feedback_comment}")
                else:
                    # Request human feedback
                    feedback_comment = await request_human_feedback(
                        ctx,
                        f"Please review {decision} decision for submission {submission_data.get('submission_id', 'Unknown')}.",
                        decision_data=decision_result
                    )
                    
                    # Store feedback
                    current_state["human_feedback_received"] = True
                    current_state["human_feedback_comment"] = feedback_comment
                    await ctx.set("state", current_state)
                    
                    logger.info(f"Human feedback received: {feedback_comment}")
                
                # Update decision based on human feedback
                if "approve" in feedback_comment.lower():
                    decision_result["outcome"] = "PASS"
                    decision_result["reason"] = "Approved by human reviewer"
                elif "reject" in feedback_comment.lower():
                    decision_result["outcome"] = "FAIL"
                    decision_result["reason"] = "Rejected by human reviewer"
                elif "more info" in feedback_comment.lower():
                    decision_result["outcome"] = "HUMAN_REVIEW"
                    decision_result["reason"] = "Human reviewer requested more information"
                
                # Mark as human reviewed and final
                decision_result["human_reviewed"] = True
                decision_result["final"] = True
                
                # Update state with final decision
                current_state["decision"] = decision_result
                await ctx.set("state", current_state)
            
            # Return final decision
            decision_messages = {
                "PASS": "Submission approved to proceed.",
                "HUMAN_REVIEW": "Submission requires additional information.",
                "FAIL": "Submission rejected."
            }
            
            logger.info(f"Final decision: {decision_result['outcome']}")
            return f"Decision: {decision_messages.get(decision_result['outcome'], decision_result['outcome'])}. Reason: {decision_result['reason']}."

        async def request_human_feedback(ctx: Context, question: str = "", decision_data: Dict[str, Any] = None) -> str:
            """
            Requests human feedback using LlamaIndex's event system.
            
            Args:
                ctx: Context object
                question: Question to ask the human reviewer
                decision_data: Optional decision data directly from the decision calculation
                
            Returns:
                Human feedback response
            """
            logger.info(f"Requesting human feedback: {question}")
            current_state = await ctx.get("state")
            
            # Use directly provided decision_data if available, otherwise fall back to context
            decision = decision_data if decision_data else current_state.get("decision", {})
            submission = current_state.get("submission_data", {})
            assessment_results = current_state.get("assessment_results", {})
            
            # Extract guideline info for readability
            raw_guidelines = current_state.get("guidelines_retrieved", {}).get("decision", "")
            lines = raw_guidelines.strip().splitlines()
            clean_lines = [line.strip() for line in lines if line.strip()]
            top_guidelines = "".join(clean_lines[:10])  # Show only top 10 lines
            guidelines_info = top_guidelines or "No specific guidelines retrieved"
            
            # Extract decision details
            outcome = decision.get("outcome", "Unknown")
            reason = decision.get("reason", "No reason provided")
            
            # Add compliance issues if present
            compliance_issues_text = ""
            hazard_issues = assessment_results.get("hazard", {}).get("issues", [])
            vulnerability_issues = assessment_results.get("vulnerability", {}).get("issues", [])
            
            if hazard_issues or vulnerability_issues:
                compliance_issues_text = "Compliance Issues:"
                if hazard_issues:
                    compliance_issues_text += "Hazard Issues:" + "".join([f"- {issue}" for issue in hazard_issues]) + ""
                if vulnerability_issues:
                    compliance_issues_text += "Vulnerability Issues:" + "".join([f"- {issue}" for issue in vulnerability_issues])
            
            # Compose human-readable question
            detailed_question = f"""
🚨 HUMAN REVIEW REQUIRED 🚨

🔍 Issue:
{question}

📄 Submission:
- ID: {submission.get('submission_id')}
- Insured: {submission.get('insured_name')}
- Broker: {submission.get('broker_name')}
- Property: {submission.get('property_details', {}).get('building_type')} at {submission.get('property_details', {}).get('address', 'Address missing')}

📊 Assessment Summary:
- Hazard Check: {"Compliant" if assessment_results.get('hazard', {}).get('compliant', False) else "Non-compliant"}
- Vulnerability Check: {"Compliant" if assessment_results.get('vulnerability', {}).get('compliant', False) else "Non-compliant"}{compliance_issues_text}

🧠 System Decision: {outcome}
📌 Reason: {reason}

📚 Key Guideline Excerpts:
{guidelines_info}

💬 Please type your decision ('approve', 'reject', 'request more info') or provide specific guidance:
"""

            # Request human feedback using LlamaIndex's event system
            response = await ctx.wait_for_event(
                HumanResponseEvent,
                waiter_id=question,
                waiter_event=InputRequiredEvent(
                    prefix=detailed_question,
                    user_name="Underwriter",
                ),
                requirements={"user_name": "Underwriter"},
            )
            
            # Record the feedback in context
            if "human_feedback" not in current_state:
                current_state["human_feedback"] = {}
            
            current_state["human_feedback"]["requested"] = True
            current_state["human_feedback"]["timestamp"] = datetime.now().isoformat()
            current_state["human_feedback"]["comment"] = response.response
            current_state["human_feedback"]["status"] = "completed"
            
            await ctx.set("state", current_state)
            
            return response.response
        
        # Communication Agent Tools
        async def send_notification(ctx: Context) -> str:
            """
            Formats and sends notification email based on the decision using appropriate templates.
            
            Returns:
                Confirmation of notification prepared
            """
            logger.info("Starting notification preparation")
            current_state = await ctx.get("state")
            decision = current_state.get("decision", {})
            
            if not decision.get("final", False):
                return "Cannot send notification for a non-final decision."
            
            # Get decision details
            outcome = decision.get("outcome", "UNKNOWN")
            reason = decision.get("reason", "No reason provided")
            submission = current_state.get("submission_data", {})
            
            try:
                # Select appropriate template based on outcome
                if outcome == "PASS":
                    # Use Example 1 (GreenTech) as template
                    email_content = format_approval_email(submission, reason)
                elif outcome == "HUMAN_REVIEW":
                    # Use Example 2 (Coffee Haven) as template - request for more info
                    email_content = format_human_review_email(submission, reason)
                else:  # FAIL
                    # Use Example 3 (Parsian Hotel) as template - rejection
                    email_content = format_rejection_email(submission, reason)
                
                # Log the email content
                logger.info(f"Email content prepared for {outcome} decision")
                
                # Store notification in state
                notification = {
                    "recipient": "distribution_team@company.com",
                    "content": email_content,
                    "sent": True,
                    "timestamp": datetime.now().isoformat()
                }
                
                current_state["notification"] = notification
                current_state["workflow_complete"] = True
                await ctx.set("state", current_state)
                
                return f"Notification email prepared for decision: {outcome}"
            except Exception as e:
                logger.error(f"Error preparing notification: {str(e)}")
                return f"Error preparing notification: {str(e)}"
            
        def format_approval_email(submission: Dict[str, Any], reason: str) -> str:
            """Format approval email using Example 1 (GreenTech) template"""
            broker_name = submission.get("broker_name", "")
            insured_name = submission.get("insured_name", "")
            property_details = submission.get("property_details", {})
            
            # Format using the structure from Example 1
            email_content = f"""
        {broker_name}

        {datetime.now().strftime("%d %B %Y")}

        Lloyd's Insurance

        100 Fenchurch Street, London, EC3M 5JD

        Dear Underwriter,

        Re: Request for Property Insurance Quote for {insured_name}

        We are pleased to inform you that the submission for {insured_name} has been reviewed and approved to proceed to quotation.

        **Property Information:**

        - Location: {property_details.get("address", "")}
        - Type: {property_details.get("building_type", "")}
        - Construction: {property_details.get("construction", "")}
        - Surface Area: {property_details.get("area_sqm", "")} m²
        - Occupancy: {property_details.get("occupancy", "")}

        Decision: Approved to proceed to quotation
        Reason: {reason}

        Please proceed with the quotation process.

        Sincerely,

        Underwriting AI Assistant
        """
            return email_content

        def format_human_review_email(submission: Dict[str, Any], reason: str) -> str:
            """Format email requesting more information using Example 2 (Coffee Haven) template"""
            broker_name = submission.get("broker_name", "")
            insured_name = submission.get("insured_name", "")
            property_details = submission.get("property_details", {})
            
            # Format using the structure from Example 2 (incomplete info email)
            email_content = f"""
        {broker_name}

        {datetime.now().strftime("%d %B %Y")}

        Lloyd's Insurance

        100 Fenchurch Street, London, EC3M 5JD

        Dear Broker,

        Re: Request for Additional Information for {insured_name}

        We are reviewing your submission for {insured_name}, but require additional information before we can proceed.

        **Property Information Review:**

        The following information is missing or requires clarification:
        - {reason}

        Please provide the requested information at your earliest convenience so we can continue processing this submission.

        **Current Property Information:**

        - Type: {property_details.get("building_type", "")}
        - Construction: {property_details.get("construction", "")}
        - Surface Area: {property_details.get("area_sqm", "")} m²
        - Occupancy: {property_details.get("occupancy", "")}

        We look forward to your response and are available to discuss any questions you may have.

        Sincerely,

        Underwriting AI Assistant
        """
            return email_content

        def format_rejection_email(submission: Dict[str, Any], reason: str) -> str:
            """Format rejection email using Example 3 (Parsian Hotel) template structure"""
            broker_name = submission.get("broker_name", "")
            insured_name = submission.get("insured_name", "")
            property_details = submission.get("property_details", {})
            
            # Format using the structure from Example 3 but adapted for rejection
            email_content = f"""
        {broker_name}

        {datetime.now().strftime("%d %B %Y")}

        Lloyd's Insurance

        100 Fenchurch Street, London, EC3M 5JD

        Dear Broker,

        Re: Submission Declined for {insured_name}

        After careful review, we regret to inform you that we are unable to proceed with the submission for {insured_name}.

        **Reason for Decline:**

        {reason}

        **Property Information Reviewed:**

        - Location: {property_details.get("address", "")}
        - Type: {property_details.get("building_type", "")}
        - Construction: {property_details.get("construction", "")}
        - Surface Area: {property_details.get("area_sqm", "")} m²
        - Occupancy: {property_details.get("occupancy", "")}

        Please note that this decision is based on our current underwriting guidelines. If you have additional information that might affect this decision, please feel free to contact us.

        Sincerely,

        Underwriting AI Assistant
        """
            return email_content
        
        # Create Hazard Agent
        self.hazard_agent = FunctionAgent(
            name="HazardClassificationAgent",
            description="Evaluates if properties comply with hazard classification guidelines.",
            system_prompt=(
                "You are a specialist in evaluating property hazard compliance for insurance purposes. "
                "You analyze building types, construction materials, and occupancy types to determine if they comply with guidelines. "
                "First, query the underwriting guidelines for hazard-related criteria. "
                "Then, check if the submission complies with these guidelines. "
                "Be thorough and precise in your assessments, looking for any violations or issues. "
                "After completing your assessment, hand off to the VulnerabilityAssessmentAgent."
            ),
            llm=self.llm,
            tools=[evaluate_hazard_classification, query_hazard_guidelines],
            can_handoff_to=["VulnerabilityAssessmentAgent"]
        )

        # Create Vulnerability Agent
        self.vulnerability_agent = FunctionAgent(
            name="VulnerabilityAssessmentAgent",
            description="Evaluates if properties comply with vulnerability guidelines.",
            system_prompt=(
                "You are a specialist in evaluating property vulnerabilities for insurance purposes. "
                "You assess security systems, fire protection measures, and other safety features to determine if they comply with guidelines. "
                "First, query the underwriting guidelines for vulnerability-related criteria. "
                "Then, check if the submission complies with these guidelines. "
                "Be thorough in identifying any non-compliance issues. "
                "After completing your assessment, hand off to the DecisionAgent."
            ),
            llm=self.llm,
            tools=[evaluate_vulnerability, query_vulnerability_guidelines],
            can_handoff_to=["DecisionAgent"]
        )

        # Create Decision Agent (continued)
        self.decision_agent = FunctionAgent(
            name="DecisionAgent",
            description="Makes binary Pass/Fail/Human decisions based on compliance checks.",
            system_prompt=(
                "You are a decision specialist for insurance underwriting. "
                "You analyze assessment results to determine whether to approve, reject, or request human review. "
                "First, check that all required assessments (hazard, vulnerability) have been completed. "
                "Then, make a binary decision: "
                "PASS if all guidelines are complied with, "
                "FAIL if any guideline is violated, "
                "HUMAN_REVIEW if information is missing or special review is needed. "
                "After making your decision, hand off to the CommunicationAgent."
            ),
            llm=self.llm,
            tools=[make_decision, query_decision_guidelines, request_human_feedback],
            can_handoff_to=["CommunicationAgent"]
        )

        # Create Communication Agent
        self.communication_agent = FunctionAgent(
            name="CommunicationAgent",
            description="Formats and sends notifications based on the decision.",
            system_prompt=(
                "You are a communication specialist for insurance operations. "
                "You create clear, appropriate notifications using email templates to inform stakeholders about underwriting decisions. "
                "For approvals, use the GreenTech example format. "
                "For information requests, use the Coffee Haven example format. "
                "For rejections, use the Parsian Hotel example format. "
                "Ensure all relevant information is included in the appropriate format."
            ),
            llm=self.llm,
            tools=[send_notification]
        )

    def create_workflow(self, submission_data: Dict[str, Any]) -> AgentWorkflow:
        """
        Create the workflow with agents
        
        Args:
            submission_data: The insurance submission data to evaluate
            
        Returns:
            AgentWorkflow object ready to run
        """
        # Create the workflow with the essential agents (no CAT agent)
        workflow = AgentWorkflow(
            agents=[
                self.hazard_agent,
                self.vulnerability_agent,
                self.decision_agent,
                self.communication_agent
            ],
            # Start with hazard agent
            root_agent=self.hazard_agent.name,
            initial_state={
                "submission_data": submission_data,
                "assessment_results": {},
                "completed_assessments": [],
                "guidelines_retrieved": {},
                "decision": {},
                "notification": {},
                "workflow_complete": False
            }
        )
        
        logger.info("Agent workflow created successfully")
        return workflow
        
    async def run_workflow(self, submission_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Run the multi-agent workflow with human-in-the-loop capability

        Args:
            submission_data: The insurance submission data to evaluate

        Returns:
            The final state after workflow completion
        """
        workflow = self.create_workflow(submission_data)

        logger.info("Starting workflow execution...")

        # Run the workflow with a more explicit message
        handler = workflow.run(
            user_msg=(
                "Please process this insurance submission through the binary compliance workflow. "
                "For each step, first query the relevant underwriting guidelines, then perform your compliance check. "
                "Begin with the hazard classification, then evaluate vulnerability, "
                "make a decision, and finally send the appropriate notification."
            )
        )

        # Stream the output to see progress
        current_agent = None

        async for event in handler.stream_events():
            # Check if workflow is complete after each step
            current_state = await handler.ctx.get("state")
            if current_state.get("workflow_complete", False):
                logger.info(
                    "Workflow has been marked as complete, stopping further processing."
                )
                break

            if (
                hasattr(event, "current_agent_name")
                and event.current_agent_name != current_agent
            ):
                current_agent = event.current_agent_name
                logger.info(f"==== Agent: {current_agent} ====")

            # Handle human input requests
            if isinstance(event, InputRequiredEvent):
                print("\n" + "=" * 50)
                print("\nHUMAN REVIEW REQUIRED")
                print("=" * 50)
                user_input = input(event.prefix + "\n> ")
                print("=" * 50 + "\n")

                # Send the human response back to the workflow
                handler.ctx.send_event(
                    HumanResponseEvent(
                        response=user_input,
                        user_name=event.user_name,
                    )
                )

            # Check if a final decision has been made
            if (
                current_state.get("decision", {}).get("final", False)
                and current_agent == "CommunicationAgent"
            ):
                logger.info(
                    "Final decision has been made and notification sent. Workflow will complete."
                )
                current_state["workflow_complete"] = True
                await handler.ctx.set("state", current_state)

        # Get the final state
        final_state = await handler.ctx.get("state")

        # Log the results
        logger.info("Workflow execution completed.")
        logger.info(
            f"Decision: {final_state.get('decision', {}).get('outcome', 'No decision')}"
        )
        logger.info(
            f"Reason: {final_state.get('decision', {}).get('reason', 'No reason provided')}"
        )

        # Return the final state
        return final_state
        

    def summarize_results(self, final_state: Dict[str, Any]) -> str:
        """
        Generate a human-readable summary of the workflow results

        Args:
            final_state: The final state after workflow completion

        Returns:
            Formatted string with summary information
        """
        decision = final_state.get("decision", {})
        submission = final_state.get("submission_data", {})

        summary = []
        summary.append("\n====== INSURANCE SUBMISSION ASSESSMENT RESULTS ======")
        summary.append(f"Submission ID: {submission.get('submission_id', 'Unknown')}")
        summary.append(f"Insured: {submission.get('insured_name', 'Unknown')}")

        # Add property details
        property_details = submission.get("property_details", {})
        if property_details:
            summary.append("\nPROPERTY DETAILS:")
            summary.append(f"- Address: {property_details.get('address', 'Unknown')}")
            summary.append(
                f"- Building Type: {property_details.get('building_type', 'Unknown')}"
            )
            summary.append(
                f"- Construction: {property_details.get('construction', 'Unknown')}"
            )
            summary.append(
                f"- Year Built: {property_details.get('year_built', 'Unknown')}"
            )
            summary.append(
                f"- Sprinklers: {'Yes' if property_details.get('sprinklers', False) else 'No'}"
            )

        # Add completed assessments
        summary.append("\nCOMPLETED ASSESSMENTS:")
        completed = ", ".join(final_state.get("completed_assessments", []))
        summary.append(f"- {completed}")

        # Add compliance summary
        summary.append("\nCOMPLIANCE SUMMARY:")
        assessment_results = final_state.get("assessment_results", {})

        hazard_result = assessment_results.get("hazard", {})
        summary.append(
            f"- Hazard Compliance: {'Compliant' if hazard_result.get('compliant', False) else 'Non-compliant'}"
        )
        if not hazard_result.get("compliant", False) and hazard_result.get(
            "issues", []
        ):
            summary.append(f"  Issues: {'; '.join(hazard_result.get('issues', []))}")

        vulnerability_result = assessment_results.get("vulnerability", {})
        summary.append(
            f"- Vulnerability Compliance: {'Compliant' if vulnerability_result.get('compliant', False) else 'Non-compliant'}"
        )
        if not vulnerability_result.get(
            "compliant", False
        ) and vulnerability_result.get("issues", []):
            summary.append(
                f"  Issues: {'; '.join(vulnerability_result.get('issues', []))}"
            )

        # Add decision
        outcome = decision.get("outcome", "Unknown")
        reason = decision.get("reason", "No reason provided")

        summary.append(f"\nDECISION: {outcome}")
        summary.append(f"Reason: {reason}")

        # Show human feedback if available
        human_feedback = final_state.get("human_feedback", {})
        if human_feedback:
            summary.append("\nHUMAN FEEDBACK:")
            summary.append(f"Status: {human_feedback.get('status', 'Not provided')}")
            summary.append(f"Comment: {human_feedback.get('comment', 'No comment')}")

        # Add notification details if available
        notification = final_state.get("notification", {})
        if notification:
            summary.append("\nNOTIFICATION STATUS:")
            summary.append(f"Recipient: {notification.get('recipient', 'Unknown')}")
            summary.append(
                f"Sent: {'Yes' if notification.get('sent', False) else 'No'}"
            )
            summary.append(f"Timestamp: {notification.get('timestamp', 'Unknown')}")

        summary.append("\n===========================================")

        return "\n".join(summary)
                    

In [8]:
def create_greentech_submission() -> Dict[str, Any]:
    """
    Create a submission dictionary for GreenTech Solutions Ltd.
    Based on Example 1 email (clean pass case)
    """
    return {
        "submission_id": "GTL20250424",
        "broker_name": "ABC Insurance Brokers",
        "insured_name": "GreenTech Solutions Ltd.",
        "property_details": {
            "address": "55 Tech Drive, London, EC1A 1BB",
            "building_type": "Commercial Office Building",
            "construction": "Steel frame with brick exterior",
            "year_built": 2010,
            "area_sqm": 780,
            "stories": 2,
            "occupancy": "Office space for 50 employees",
            "sprinklers": True,
            "alarm_system": "24/7 CCTV monitoring",
        },
        "coverage": {
            "building_value": 2500000.0,
            "contents_value": 1000000.0,
            "business_interruption": 500000.0,
            "deductible": 1000.0,
        },
        "risk_assessment": {
            "fire_hazards": "Sprinkler system installed, regular fire drills",
            "natural_disasters": "Low flood risk area, not located near seismic fault lines",
            "security_measures": "24/7 CCTV monitoring, keycard access control",
        },
        "financial_info": {"property_value": 2500000.0, "business_revenue": 8000000.0},
    }


def create_coffee_heaven_submission() -> Dict[str, Any]:
    """
    Create a submission dictionary for Coffee Heaven Ltd.
    Based on Example 2 email (missing information case)
    """
    return {
        "submission_id": "CHL20250424",
        "broker_name": "XYZ Insurance Services",
        "insured_name": "Coffee Heaven Ltd.",
        "property_details": {
            "address": "",  # Missing address to trigger human review
            "building_type": "Commercial Coffee Shop and Retail Store",
            "construction": "Traditional brick and mortar",
            "year_built": 1998,
            "area_sqm": 250,
            "stories": 2,
            "occupancy": "Coffee shop production on the ground floor, retail space on the first floor",
            "sprinklers": True,
            "alarm_system": "Alarm system, 24/7 security monitoring",
        },
        "coverage": {
            "building_value": 1800000.0,
            "contents_value": 750000.0,
            "business_interruption": 250000.0,
            "deductible": 750.0,
        },
        "risk_assessment": {
            "fire_hazards": "Industrial ovens in use, sprinkler system in place",
            "natural_disasters": "Low flood risk, located in an area not prone to earthquakes",
            "security_measures": "Alarm system, 24/7 security monitoring",
        },
        "financial_info": {"property_value": 1800000.0, "business_revenue": 500000.0},
    }


def create_parsian_hotel_submission() -> Dict[str, Any]:
    """
    Create a submission dictionary for Parsian Evin Hotel Ltd.
    Based on Example 3 email (sanctions/rejection case)
    """
    return {
        "submission_id": "PEH20250424",
        "broker_name": "Prime Insurance Brokers",
        "insured_name": "Parsian Evin Hotel Ltd.",
        "property_details": {
            "address": "No. 45, Evin Street, Tehran, Iran",  # Sanctions trigger
            "building_type": "Hotel",
            "construction": "Modern design, reinforced concrete and steel",
            "year_built": 2010,
            "area_sqm": 11500,
            "stories": 5,
            "occupancy": "150-room hotel, luxury restaurant, and conference facilities",
            "sprinklers": True,
            "alarm_system": "CCTV surveillance, 24/7 security personnel",
        },
        "coverage": {
            "building_value": 800000000000.0,  # in IRR
            "contents_value": 400000000000.0,
            "business_interruption": 200000000000.0,
            "deductible": 500000000.0,
        },
        "risk_assessment": {
            "fire_hazards": "Fire alarm and sprinkler system in all rooms, fire exits clearly marked",
            "natural_disasters": "Low flood risk, not located in an earthquake-prone area, occasional sandstorms",
            "security_measures": "CCTV surveillance, 24/7 security personnel, secure entry systems",
        },
        "financial_info": {
            "property_value": 1000000000000.0,
            "business_revenue": 300000000000.0,
        },
    }

In [9]:
# Example usage


async def main():
    """Run the insurance agent system with example data"""

    # Create the system
    system = InsuranceAgentSystem()

    # Use the three example submissions
    greentech = create_greentech_submission()  # Should PASS
    coffee_heaven = (
        create_coffee_heaven_submission()
    )  # Should trigger HUMAN_REVIEW for missing address
    parsian_hotel = create_parsian_hotel_submission()  # Should FAIL for sanctions

    # Choose which one to run (comment/uncomment as needed)
    # submission_data = greentech
    submission_data = coffee_heaven
    # submission_data = parsian_hotel

    # Run the workflow
    final_state = await system.run_workflow(submission_data)

    # Show summary
    print(system.summarize_results(final_state))

    return final_state


if __name__ == "__main__":
    asyncio.run(main())

2025-04-28 17:56:17,255 - InsuranceAgentSystem - INFO - LLM and embedding models initialized
2025-04-28 17:56:17,256 - InsuranceAgentSystem - INFO - Loading existing index from storage/guidelines_index
2025-04-28 17:56:17,257 - llama_index.core.storage.kvstore.simple_kvstore - DEBUG - Loading llama_index.core.storage.kvstore.simple_kvstore from storage/guidelines_index/docstore.json.
2025-04-28 17:56:17,257 - fsspec.local - DEBUG - open file: /home/azureuser/projects/insureflow/experiments/notebooks/anna/storage/guidelines_index/docstore.json
2025-04-28 17:56:17,258 - InsuranceAgentSystem - ERROR - Error setting up guidelines vector store: [Errno 2] No such file or directory: '/home/azureuser/projects/insureflow/experiments/notebooks/anna/storage/guidelines_index/docstore.json'
2025-04-28 17:56:17,259 - InsuranceAgentSystem - ERROR - Traceback: Traceback (most recent call last):
  File "/tmp/ipykernel_58730/3153839684.py", line 63, in setup_guidelines_vector_store
    storage_context =



HUMAN REVIEW REQUIRED


2025-04-28 17:57:19,737 - InsuranceAgentSystem - INFO - Human feedback received: approve
2025-04-28 17:57:19,738 - InsuranceAgentSystem - INFO - Final decision: PASS
2025-04-28 17:57:19,745 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'api-key': '<redacted>'}, 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a decision specialist for insurance underwriting. You analyze assessment results to determine whether to approve, reject, or request human review. First, check that all required assessments (hazard, vulnerability) have been completed. Then, make a binary decision: PASS if all guidelines are complied with, FAIL if any guideline is violated, HUMAN_REVIEW if information is missing or special review is needed. After making your decision, hand off to the CommunicationAgent.'}, {'role': 'user', 'content': "Current state:\n{'submission_data': {'submission_id': 'CHL20250424', '

2025-04-28 17:57:20,071 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Transfer-Encoding', b'chunked'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'apim-request-id', b'95793369-db9b-4bb0-b5ed-4fdaa88dc7e7'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubDomains; preload'), (b'x-content-type-options', b'nosniff'), (b'x-ms-region', b'UK South'), (b'x-ratelimit-remaining-requests', b'2325'), (b'x-ratelimit-limit-requests', b'2330'), (b'x-ratelimit-remaining-tokens', b'226689'), (b'x-ratelimit-limit-tokens', b'233000'), (b'cmp-upstream-response-duration', b'240'), (b'x-accel-buffering', b'no'), (b'x-ms-rai-invoked', b'true'), (b'x-envoy-upstream-service-time', b'220'), (b'x-request-id', b'93bb3cda-44f5-4ad6-b41b-706b732dd576'), (b'ms-azureml-model-time', b'218'), (b'x-ms-client-request-id', b'95793369-db9b-4bb0-b5ed-4fdaa88dc7e7'), (b'azureml-model-session', b'v20250411-1-167922813'), (b'x-ms-deployment


====== INSURANCE SUBMISSION ASSESSMENT RESULTS ======
Submission ID: CHL20250424
Insured: Coffee Heaven Ltd.

PROPERTY DETAILS:
- Address: 
- Building Type: Commercial Coffee Shop and Retail Store
- Construction: Traditional brick and mortar
- Year Built: 1998
- Sprinklers: Yes

COMPLETED ASSESSMENTS:
- hazard, vulnerability

COMPLIANCE SUMMARY:
- Hazard Compliance: Compliant
- Vulnerability Compliance: Compliant

DECISION: PASS
Reason: Approved by human reviewer

HUMAN FEEDBACK:
Status: completed
Comment: approve



2025-04-28 17:57:21,002 - httpcore.http11 - DEBUG - receive_response_body.complete
2025-04-28 17:57:21,002 - httpcore.http11 - DEBUG - response_closed.started
2025-04-28 17:57:21,003 - httpcore.http11 - DEBUG - response_closed.complete
2025-04-28 17:57:21,005 - InsuranceAgentSystem - INFO - Starting notification preparation
2025-04-28 17:57:21,006 - InsuranceAgentSystem - INFO - Email content prepared for PASS decision
2025-04-28 17:57:21,013 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'api-key': '<redacted>'}, 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a communication specialist for insurance operations. You create clear, appropriate notifications using email templates to inform stakeholders about underwriting decisions. For approvals, use the GreenTech example format. For information requests, use the Coffee Haven example format. For rejections, use the Parsian Hot